In [1]:
import os
import psycopg2
import pandas as pd
import numpy as np
import sqlalchemy
import re
from sqlalchemy import create_engine 

In [2]:
mot_passe = os.environ.get('pg_psw')


In [3]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database Testrome''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()


Base de données créée avec succès !


In [4]:
conn = psycopg2.connect(
   database="testrome", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

In [5]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

In [6]:
sql_creer_table_calendar = """
    CREATE TABLE IF NOT EXISTS calendar (
    date_naissance date,
    date_mort date
    );
"""

In [7]:
creer_table(conn, sql_creer_table_calendar)

La table a été crée avec succès


In [16]:
data = {'date_naissance':  ['0100-01-01', '-0100-01-01'],
        'date_mort': ['0200-01-01', '0000-01-01']
        }

df = pd.DataFrame(data, columns = ['date_naissance','date_mort'])


In [17]:
df

,date_naissance,date_mort
0,0100-01-01,0200-01-01
1,-0100-01-01,0000-01-01


In [18]:
df['date_naissance']

0     0100-01-01
1    -0100-01-01
Name: date_naissance, dtype: object

In [19]:
df = df.replace(r"^-([0-9]*-[0-9]*-[0-9]*)$", r"\1BC", regex = True)
df = df.replace(r"^0000(-[0-9]*-[0-9]*)$", r"0001\1BC", regex = True)

In [20]:
df

,date_naissance,date_mort
0,0100-01-01,0200-01-01
1,0100-01-01BC,0001-01-01BC


In [21]:
sql_insert_calendar = """INSERT INTO calendar (date_naissance, date_mort) VALUES (%s, %s)"""

In [22]:
conn = psycopg2.connect(
   database="testrome", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(sql_insert_calendar, tuple(row))